In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import gym # openai to render environment for agent to take action in and receive feedback in
import gym.wrappers
import random
import numpy as np
from collections import deque # for memory of agent
from tensorflow import keras
from keras.models import Sequential # to build neural network for aproximating optimal Q
from keras.layers import Dense # theta weights - weights and bias of neurons in between layers of dense network
from keras.optimizers import Adam # stochastic gradient descent

#### Set Parameters

In [2]:
env = gym.make("MountainCar-v0")

In [3]:
# hyperparameters related to size of state and size of actions
state_size = env.observation_space.shape[0]
state_size

2

In [4]:
action_size = env.action_space.n
action_size

3

In [5]:
# hyperparameter for gradient descent (vary by powers of 2)
batch_size = 32
n_episodes = 8000
output_dir = "model_output/MountainCar"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [6]:
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        # our agent's memory
        # prevents us from going over every single event that's happened. Too Slow!
        self.memory = deque(maxlen=200000)
        
        '''hyperparameters''' 
        self.gamma = 0.99 # discount factor of future rewards
        
        # two modes of action: 
            # exploitation: take best possible action based on whats been learned
            # exploration: to explore the environment more and find new actions
        self.epsilon = 1.0 # initial exploration rate of agent 
        self.epsilon_decay = 0.85 # slowly shifts from exploring to exploitation
        self.epsilon_min = 0.00001 # lowest exploration percent can decay to
        
        self.learning_rate = 0.001251 # stochastic gradient descent step size
        
        self.model = self._build_model() # ensures private method can only be used by this particular instance of a class
        self.target_model = self._build_model()
        
        self.update_target_model()
        
    def _build_model(self): # where we define dense neural network for approx Q*
        model = Sequential()
        
        model.add(Dense(24, input_dim = self.state_size, activation="relu")) # hidden layer
        model.add(Dense(48, activation="relu")) # hidden layer        
        
        # output layer: as many neurons as possible actions
        model.add(Dense(self.action_size, activation="linear")) # reason for linear: we are directly modeling actions, no abstract probability
        model.compile(loss="mse", optimizer=Adam(lr=self.learning_rate)) # mse works for this agent, but cross-entropy might work better for others. mse is not usually first choice
        
        return model
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
    
    # REALLY IMPORTANT: takes in state, action, reward, and next_state at current time step
    # to model what will happen in next_state and what reward we can expect to receive 
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    # figuring out what action to take based on state
    def act(self, state):
        '''
        explore randomly or exploit information accrued in dense network
        as epsilon decays exploit will be more likely
        '''
        
        # explore
        if np.random.rand(1) <= self.epsilon:
            return random.randrange(self.action_size)
        
        #exploit
        act_values = self.model.predict(state) # use theta weights and predict method on our model inside the agent to guess best course of action to maz future reward
        return np.argmax(act_values[0])
    
    # bulk of agent defs
    def replay(self, batch_size):
        
        # samples from the deque of memory
        minibatch = random.sample(self.memory, batch_size)
        
        states = []
        targets = []
        
        for state, action, reward, next_state, done in minibatch:
            # if reached max time steps or by dying, then done
            target = reward
            if not done:
                # reward plus estimates of future reward using neural network and next state
                target = (reward + self.gamma * np.amax(self.target_model.predict(next_state)[0]))
            
            # map maximized future reward to current reward with theta
            target_f= self.model.predict(state) # use neural network to estimate target given current state
            target_f[0][action]= target # map to future state
            
            states.append(state[0])
            targets.append(target_f[0])
            
        # train model with X: current state, Y: future reward, 
        self.model.fit(np.array(states), np.array(targets), epochs=1, verbose=0)

    def load(self, name):
        self.model.load_weights(name)
    
    def save(self, name):
        self.model.save_weights(name)

#             # if reached max time steps or by dying, then done
#             target = reward
#             if not done:
#                 # reward plus estimates of future reward using neural network and next state
#                 target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            
#             # map maximized future reward to current reward with theta
#             target_f = self.model.predict(state) # use neural network to estimate target given current state
#             target_f[0][action] = target # map to future state
            



In [7]:
agent = Agent(state_size, action_size)

In [ ]:
done = False

scores_memory = deque(maxlen=100)

for e in range(n_episodes):
    
    # start each episode at beginning state
    state = env.reset()
    
    # transpose state to fit nicely with DL network
    state = np.reshape(state, [1, state_size])
    
    # iterate over time steps of game
    for time in range(5000):
        
        if e % 50 == 0:
            env.render()
        
        action = agent.act(state)
        
        # returned values from taking a step forward
        next_state, reward, done, _ = env.step(action)
        
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
        
        # run default prediction model
        if len(agent.memory)>batch_size:
            agent.replay(batch_size)
        
        # moved into next state
        state = next_state
        
        if done:
            scores_memory.append(time)
            scores_avg = np.mean(scores_memory) * - 1.0
            print('episode: {}/{}, score: {}, e {:.2}, help: {}, reward: {}, 100score avg: {}'.format(e, n_episodes, time, agent.epsilon, state, reward, scores_avg))
            break
            
    agent.update_target_model()

    if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay
        
    if e % 50 == 0:
        agent.save(output_dir + 'weights_final' + '{:04d}'.format(e) + ".hdf5")

episode: 0/8000, score: 199, e 1.0, help: [[-0.57948268  0.01260537]], reward: -1.0, 100score avg: -199.0
episode: 1/8000, score: 199, e 0.85, help: [[-0.55856871 -0.00198235]], reward: -1.0, 100score avg: -199.0
episode: 2/8000, score: 199, e 0.72, help: [[-0.54974823  0.01699448]], reward: -1.0, 100score avg: -199.0
episode: 3/8000, score: 199, e 0.61, help: [[-0.57335211  0.01094781]], reward: -1.0, 100score avg: -199.0
episode: 4/8000, score: 199, e 0.52, help: [[-0.24606916  0.02434657]], reward: -1.0, 100score avg: -199.0
episode: 5/8000, score: 199, e 0.44, help: [[-0.51362194  0.0121166 ]], reward: -1.0, 100score avg: -199.0
episode: 6/8000, score: 199, e 0.38, help: [[-0.53800659  0.01649658]], reward: -1.0, 100score avg: -199.0
episode: 7/8000, score: 199, e 0.32, help: [[-0.68426201  0.00870184]], reward: -1.0, 100score avg: -199.0
episode: 8/8000, score: 199, e 0.27, help: [[-0.54446483 -0.02584037]], reward: -1.0, 100score avg: -199.0
episode: 9/8000, score: 199, e 0.23, h